In [1]:
import os
import os
import ipykernel
import pathlib
nb_path = pathlib.Path(os.path.abspath(ipykernel.__file__)).parent.resolve(strict=True)
current_working_directory = os.getcwd()
print(f"nb_path = {nb_path}, current_working_directory={current_working_directory}")
import sys, os
sys.path.append(os.path.dirname(current_working_directory))
from langchain import PromptTemplate, LLMChain
from models.glm_llm import glm_llm as glm
llm = glm()


nb_path = /sata-data/apps/micromamba/envs/chatdoc/lib/python3.9/site-packages/ipykernel, current_working_directory=/sata-data/apps/llm/chatdoc-server/chains


In [ ]:
question = "AI知识库用到的开源软件有哪些"
context = "以OpenAI公司GPT4为代表的⼤语⾔模型（LLM）涌现出很多强大的能力，利用LLM搭建具有语义检索和知识总结的新⼀代知识库成为了可能。LLM在铁路企业落地应⽤⾸先要解决⼤模型如何在本地部署的问题。今年以来，随着60亿650亿参数的LLaMA开源基础语⾔模型的快速发展，针对这类可以负担得起的大模型进行微调来完成企业特定任务成为新趋势。AI知识库⽤到的开源软件包括向量数据库和⼤模型两部分。向量数据库主要解决对⽂档进⾏语义检索，然后利⽤LLM对检索出来的结果进⾏提炼和总结并输出。我们选⽤的Milvus向量数据库和ChatGLM-6B⼤语⾔模型都是国内⽐较出⾊的开源软件，对中文语料支持较好。ChatGLM-6B对算⼒有⼀定要求，最好是支持Tensor Core的高性能显卡，在消费级显卡中，英伟达的RTX4090有24G显存，能满足Transformer模型的最低要求。目前京东上的公开报价约1.6w⼀张。在现阶段我们建议配置⼀台双RTX4090的工作站"
prompt_template = """基于以下已知信息，简洁和专业的来回答用户的问题。
                    如果无法从中得到答案，请说 "根据已知信息无法回答该问题" 或 "没有提供足够的相关信息"，不允许在答案中添加编造成分，答案请使用中文。
                    已知内容:
                    {context}
                    问题:
                    {question}"""
prompt = PromptTemplate(template=prompt_template,input_variables=["context", "question"])

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)
answer = llm_chain.predict(context = context, question = question)
print(answer)

In [2]:
from langchain.base_language import BaseLanguageModel
from langchain.chains.combine_documents.base import BaseCombineDocumentsChain
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain.chains.combine_documents.refine import RefineDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.chains.summarize import map_reduce_prompt, refine_prompts, stuff_prompt, _load_map_reduce_chain, _load_refine_chain, _load_stuff_chain, LoadingCallable
from langchain.prompts.base import BasePromptTemplate
from typing import Optional, Any, Mapping
from prompts import stuff_prompt, refine_prompt, mapreduce_prompt
from loader.pdf_loader import loadDocsFromPDF
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [3]:
import json
# from kb_select_prompt import KB_SELECT_PROMPT_TMPL
KB_SELECT_PROMPT_TMPL = """请从下面的文本中提取关键信息,并用json格式输出,我先给你几个例子,你将严格按照例子里的json格式输出,不要添加任何其他内容
问题:请从建设部的文件中查找关于建设项目资金管理方面的内容
回答:"department':"建设部", "search_query":"建设资金管理", "knowledge_base":"建设部知识库"
====================
问题:请检索建设部下发的文件，查找关于建设资金管理的信息
回答:"department":"建设部","search_query":"建设项目资金管理", "knowledge_base":"建设部知识库"
====================
问题:请打开建设部知识库，查找关于建设资金管理的信息
回答:"department":"建设部", "search_query":"建设项目资金管理", "knowledge_base":"建设部知识库"
====================
问题:请打开建设部知识库
回答:"department":"建设部", "search_query": "", "knowledge_base":"建设部知识库"
====================
问题:在建设部的文件中有哪些关于建设资金管理的内容
回答:"department":"建设部", "search_query":"建设项目资金管理", "knowledge_base":"建设部知识库"
====================
问题:请从客运部文件中查找安全生产的资料
回答:"department":"客运部", "search_query":"安全生产", "knowledge_base":"客运部知识库"
====================
问题:请打开langchain知识库
回答:"department":"", "search_query":"", "knowledge_base":"langchain知识库"
====================
问题:请打开安全生产问答知识库
回答:"department":"", "search_query":"", "knowledge_base":"安全生产问答知识库"
====================
问题:从word文档中查找关于建设资金管理的内容
回答:"department":"", "search_query":"查找关于建设资金管理的内容", "knowledge_base":"word文档知识库"
====================
问题:搜寻word文档知识库
回答:"department":"", "search_query":"", "knowledge_base":"word文档知识库"

现在请从下面的文本中提取关键信息,直接输出结果：
{question}
"""


In [4]:
def parse_collection(question = "请从广铁集团客运部下发的管理文件中查找关于安全生产的资料"):
    prompt = PromptTemplate(template=KB_SELECT_PROMPT_TMPL,input_variables=["question"])
    llm_chain = LLMChain(
        llm=llm,
        prompt=prompt
    )
    answer = llm_chain.predict(question = question)
    sub_str = '"'
    index = answer.find(sub_str)
    answer = answer[index:] if index != -1 else answer
    class kb_selector(BaseModel):
        knowledge_base: str = Field(description="name of an knowledge_base")
        search_query: str = Field(description="query of a search")
        department: str = Field(description="department")      
    parser = PydanticOutputParser(pydantic_object=kb_selector)
    from langchain.output_parsers import OutputFixingParser
    new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
    try:
        kb = new_parser.parse(answer)
    except Exception as e:
        kb = kb_selector(knowledge_base="", search_query="", department="")
    print(f"inpout = {question}\n answer = {answer}\n knowledge_base = {kb.knowledge_base}")
    return kb

In [5]:
# parse_collection()
parse_collection(question = "打开excel知识库")
parse_collection(question = "检索''excel知识库")
parse_collection(question = "寻找excel知识库")
parse_collection(question = "从excel知识库中寻找文件操作方法")
parse_collection(question = "检索excel文件，寻找文件操作方法")
parse_collection(question = "在客运部的文件中寻找关于客运段安全生产的规定")
parse_collection(question = "在客运部的有关规定中寻找关于客运段安全生产的规定")


inpout = 打开excel知识库
 answer = "department":"", "search_query":"", "knowledge_base":"excel知识库"
 knowledge_base = 知识库
inpout = 检索''excel知识库
 answer = "department": "excel", "search_query": "", "knowledge_base": "excel知识库"
 knowledge_base = excel知识库
inpout = 寻找excel知识库
 answer = "knowledge_base": "excel知识库", "department": "电子工业部", "search_query": "Excel"
 knowledge_base = Excel知识库
inpout = 从excel知识库中寻找文件操作方法
 answer = "department": "excel", "search_query": "文件操作方法", "knowledge_base": "excel知识库"
 knowledge_base = excel知识库
inpout = 检索excel文件，寻找文件操作方法
 answer = "department": "excel", "search_query": "文件操作方法", "knowledge_base": "excel文档知识库"}
 knowledge_base = Knowledge Base
inpout = 在客运部的文件中寻找关于客运段安全生产的规定
 answer = "department": "客运部", "search_query": "安全生产", "knowledge_base": "客运部知识库"
 knowledge_base = 客运部知识库
inpout = 在客运部的有关规定中寻找关于客运段安全生产的规定
 answer = "department": "客运部", "search_query": "安全生产", "knowledge_base": "客运部知识库"
 knowledge_base = 客运部知识库


kb_selector(knowledge_base='客运部知识库', search_query='安全生产', department='客运部')